# Sistema de Webhooks y Notificaciones

Este notebook muestra como configurar y usar el sistema de notificaciones mediante webhooks.

In [ ]:
import sys
sys.path.insert(0, '..')

import asyncio
import json

from notifications.webhook import webhook_manager
from notifications.watcher import dataset_watcher

## 1. Registrar un Webhook

Los webhooks te permiten recibir notificaciones HTTP cuando hay cambios en datasets.

In [ ]:
# Ejemplo: registrar un webhook (usa tu propia URL)
# Puedes usar https://webhook.site para pruebas

ejemplo_url = "https://webhook.site/tu-uuid-aqui"  # Cambia esto

# Registrar webhook para todos los datasets
webhook = webhook_manager.register(
    webhook_url=ejemplo_url,
    event_types=["created", "updated", "deleted"]
)

print(f"Webhook registrado:")
print(json.dumps(webhook, indent=2))

In [ ]:
# Registrar webhook para un dataset especifico
webhook_especifico = webhook_manager.register(
    webhook_url=ejemplo_url,
    dataset_id="mi-dataset-id",
    event_types=["updated"]
)

print(f"Webhook especifico registrado: {webhook_especifico['id']}")

In [ ]:
# Registrar webhook para una tematica
webhook_tema = webhook_manager.register(
    webhook_url=ejemplo_url,
    theme="economia",
    event_types=["created", "updated"]
)

print(f"Webhook de tema registrado: {webhook_tema['id']}")

## 2. Listar Webhooks

In [ ]:
# Ver todos los webhooks registrados
webhooks = webhook_manager.list_all()

print(f"Total webhooks: {len(webhooks)}")
for wh in webhooks:
    print(f"\n- ID: {wh['id']}")
    print(f"  URL: {wh['url'][:50]}...")
    print(f"  Dataset: {wh.get('dataset_id', 'Todos')}")
    print(f"  Tema: {wh.get('theme', 'Todos')}")
    print(f"  Eventos: {wh['event_types']}")
    print(f"  Activo: {wh['active']}")

## 3. Probar un Webhook

In [ ]:
# Enviar notificacion de prueba
async def probar_webhook(webhook_id):
    result = await webhook_manager.trigger(
        webhook_id=webhook_id,
        event_type="updated",
        payload={
            "test": True,
            "message": "Notificacion de prueba"
        }
    )
    return result

if webhooks:
    resultado = await probar_webhook(webhooks[0]['id'])
    print(f"Resultado: {json.dumps(resultado, indent=2)}")

## 4. Vigilar Cambios en Datasets

In [ ]:
# El watcher guarda snapshots de datasets para detectar cambios

# Simular datos de un dataset
dataset_ejemplo = {
    "_about": "https://datos.gob.es/apidata/catalog/dataset/ejemplo",
    "title": {"_value": "Dataset de Ejemplo"},
    "description": {"_value": "Descripcion del dataset"},
    "modified": "2024-12-26T10:00:00Z",
    "distribution": [
        {"accessURL": "https://ejemplo.com/data.csv"},
        {"accessURL": "https://ejemplo.com/data.json"}
    ]
}

# Tomar snapshot inicial
snapshot = dataset_watcher.take_snapshot("ejemplo-dataset", dataset_ejemplo)
print(f"Snapshot creado:")
print(json.dumps(snapshot, indent=2))

In [ ]:
# Simular un cambio en el dataset
dataset_modificado = dataset_ejemplo.copy()
dataset_modificado["modified"] = "2024-12-26T15:00:00Z"
dataset_modificado["distribution"] = [
    {"accessURL": "https://ejemplo.com/data.csv"},
    {"accessURL": "https://ejemplo.com/data.json"},
    {"accessURL": "https://ejemplo.com/data.xlsx"}  # Nueva distribucion
]

# Detectar cambios
cambios = dataset_watcher.check_changes("ejemplo-dataset", dataset_modificado)
print(f"Resultado de verificacion:")
print(json.dumps(cambios, indent=2))

In [ ]:
# Listar datasets vigilados
vigilados = dataset_watcher.list_watched()
print(f"Datasets vigilados: {len(vigilados)}")
for v in vigilados:
    print(f"  - {v['dataset_id']}: ultima revision {v['last_check']}")

## 5. Flujo Completo: Detectar y Notificar

In [ ]:
async def flujo_completo(dataset_id, current_data):
    """Flujo completo de deteccion y notificacion."""
    
    # 1. Verificar cambios
    cambios = dataset_watcher.check_changes(dataset_id, current_data)
    
    if cambios['status'] == 'changed':
        print(f"Cambios detectados en {dataset_id}")
        
        # 2. Notificar a todos los webhooks relevantes
        resultados = await webhook_manager.notify_change(
            dataset_id=dataset_id,
            event_type="updated",
            changes=cambios
        )
        
        print(f"Notificaciones enviadas: {len(resultados)}")
        return resultados
    else:
        print(f"No hay cambios en {dataset_id}")
        return []

# Ejemplo de uso
# resultados = await flujo_completo("mi-dataset", datos_actuales)

## 6. Limpiar Webhooks

In [ ]:
# Eliminar un webhook
if webhooks:
    eliminado = webhook_manager.unregister(webhooks[0]['id'])
    print(f"Webhook eliminado: {eliminado}")

# Ver webhooks restantes
print(f"\nWebhooks restantes: {len(webhook_manager.list_all())}")

In [ ]:
# Limpiar snapshots de datasets
# dataset_watcher.clear_all()  # Descomentar para limpiar
# print("Todos los snapshots eliminados")

## Estructura del Payload de Webhook

Cuando se dispara un webhook, se envia un POST con este formato:

```json
{
  "event_type": "updated",
  "timestamp": "2024-12-26T15:30:00.000000",
  "webhook_id": "abc123",
  "data": {
    "dataset_id": "mi-dataset",
    "changes": {
      "status": "changed",
      "changes": [
        {"field": "modified", "previous": "...", "current": "..."}
      ]
    }
  },
  "signature": "..."  // Si WEBHOOK_SECRET esta configurado
}
```

## Fin de los Tutoriales

Has completado todos los notebooks de ejemplo. Ahora sabes como:

1. Buscar y filtrar datasets en datos.gob.es
2. Descargar y analizar datos
3. Usar las integraciones con INE, AEMET y BOE
4. Configurar webhooks para recibir notificaciones

Para mas informacion, consulta la [documentacion](../README.md).